In [84]:
import numpy  as np
import pandas as pd
import math

In [85]:
data = pd.read_csv("clean_weather.csv",index_col=0)
data = data.ffill()

In [ ]:
data.head

### Forward pass

In [27]:
np.random.seed(0)

i_weights = np.random.rand(1,5) / 5 - 0.1
h_weights = np.random.rand(5,5) /5 - 0.1
h_bias = np.random.rand(1,5) / 5 - 0.1

o_weights = np.random.rand(5,1) * 50
o_bias = np.random.rand(1,1)

In [28]:
outputs =  np.zeros(3)
hiddens = np.zeros((3,5))
prev_hidden = None
sequence = data['tmax'].tail(3).to_numpy()

for i in range(3):
    x = sequence[i].reshape(1,1)
    xi = x @ i_weights #np.dot(x,i_weights)
    
    if prev_hidden is None:
        xh = xi
    else:
        xh = xi + prev_hidden @ h_weights + h_bias
        
    xh = np.tanh(xh)
    prev_hidden = xh
    hiddens[i,] = xh
    
    xo = xh @ o_weights + o_bias
    outputs[i] = xo[0][0]
    
    

### Backward pass

In [29]:
def mse(actual,predicted):
    return np.mean((actual - predicted)**2)

def mse_grad(actual,predicted):
    return (predicted - actual)




In [30]:
actuals = np.array([70,62,75])
loss_grad = mse_grad(actuals,outputs)


In [31]:
next_hidden = None

o_weight_grad, o_bias_grad, hidden_weights_grad,hidden_bias_grad, i_weight_grad = [0]*5

for i in range(2,-1,-1):
    l_grad = loss_grad[i].reshape(1,1)
    o_weight_grad += hiddens[i][:,np.newaxis] @ l_grad
    o_bias_grad += np.mean(l_grad)
    
    o_grad = l_grad @  o_weights.T
    
    if next_hidden is None:
        h_grad = o_grad
    else:
        h_grad = o_grad + next_hidden @ h_weights.T
    
    dtanh = 1 - hiddens[i,:][np.newaxis,:]**2
    h_grad = np.multiply(h_grad,dtanh)
    
    next_hidden = h_grad
    
    if i>0 :
        hidden_weights_grad += hiddens[i-1,:][:,np.newaxis] @ h_grad
        h_bias += np.mean(h_grad)
    
    i_weight_grad += sequence[i].reshape(1,1).T @ h_grad
        
        
    
    

### Updating the weights

In [35]:
lr = 1e-6
i_weights -= i_weight_grad * lr
h_weights -= hidden_weights_grad * lr
h_bias -= hidden_bias_grad * lr
o_weights -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

## Complete implementation

#### Splitting data (for non time-series-data)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

predictors = ["tmax","tmin","rain"]
target = "tmax_tomorrow"

#following method can be used for normal splitting but since we have time series data we will use differen method
#Splitting the data into train and temp parts (temp will be converted to test and validation sets)
X = data[predictors]
Y = data[target]

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.3,
    random_state=0
)

# Second split: Split temp into validation (15%) and test (15%)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 0.5 of 30% = 15%
    random_state=0
)

# Scale the features AFTER splitting to prevent data leakage
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # only transform, don't fit
X_test_scaled = scaler.transform(X_test)    # only transform, don't fit




#### Splitting data (for time-series)

In [3]:
# Assuming 'date' is your date column and data is sorted by date
from sklearn.preprocessing import StandardScaler

predictors = ["tmax", "tmin", "rain"]
target = "tmax_tomorrow"

# Calculate split points
total_rows = len(data)
train_end = int(0.7 * total_rows)
valid_end = int(0.85 * total_rows)

# Split the data
X_train = data[predictors].iloc[:train_end]
y_train = data[target].iloc[:train_end]

X_valid = data[predictors].iloc[train_end:valid_end]
y_valid = data[target].iloc[train_end:valid_end]

X_test = data[predictors].iloc[valid_end:]
y_test = data[target].iloc[valid_end:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_valid_scaled = np.array(X_valid_scaled)
X_test_scaled = np.array(X_test_scaled)

#We dont scale target data check on google for more info on why
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [ ]:
X_train_scaled.shape

##### layer configuration

In [40]:
layer_conf = [{"type":"input","units":3},
              {"type": "rnn","hidden":4,"output":1}
              ]

##### Initializing weights

In [88]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

##### Forward pass

In [89]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

In [90]:
def mse(actual,predicted):
    return np.mean((actual - predicted)**2)

def mse_grad(actual,predicted):
    return (predicted - actual)


In [91]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

In [65]:
sequence_len = 7
layers = init_params(layer_conf)
for j in range(X_train_scaled.shape[0]-sequence_len):
        seq_x = X_train_scaled[j:(j+sequence_len),]
        seq_y = y_train[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x,layers)
        grad = mse_grad(seq_y, outputs)

In [75]:
layers = []
for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

In [ ]:
o_weight.shape

##### Training loop

In [87]:
from sklearn.preprocessing import StandardScaler
import math

# Define predictors and target
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

# Scale our data to have mean 0
scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

# Split into train, valid, test sets
np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

/home/nishantk30/miniconda3/envs/dataviz/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [92]:
epochs = 250
lr = 1e-5

layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    
    for j in range(X_train_scaled.shape[0]-sequence_len):
        seq_x = X_train_scaled[j:(j+sequence_len),]
        seq_y = y_train[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x,layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers,seq_x,lr,grad,hiddens)
        epoch_loss += mse(seq_y,outputs)
        
    
    if epochs%50 == 0:
        valid_loss = 0
        for j in range(X_valid_scaled.shape[0]-sequence_len):
            seq_x = X_train_scaled[j:(j+sequence_len)]
            seq_y = y_train[j:(j+sequence_len)]
            _, outputs = forward(seq_x,layers)
            valid_loss += mse(seq_y,outputs)
        print(f"Epoch : {epoch}, train_loss  : {epoch_loss / len(X_train_scaled)}, valid_loss : {valid_loss / len(X_valid_scaled)}")
            

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 7)

In [86]:
from sklearn.preprocessing import StandardScaler
import math

# Define predictors and target
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

# Scale our data to have mean 0
scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

# Split into train, valid, test sets
np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

/home/nishantk30/miniconda3/envs/dataviz/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [78]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

In [79]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

In [80]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

In [ ]:
epochs = 250
lr = 1e-5

layer_conf = [
    {"type":"input", "units": 3},
    {"type": "rnn", "hidden": 4, "output": 1}
]
layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    for j in range(train_x.shape[0] - sequence_len):
        seq_x = train_x[j:(j+sequence_len),]
        seq_y = train_y[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x, layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers, seq_x, lr, grad, hiddens)
        epoch_loss += mse(seq_y, outputs)

    if epoch % 50 == 0:
        sequence_len = 7
        valid_loss = 0
        for j in range(valid_x.shape[0] - sequence_len):
            seq_x = valid_x[j:(j+sequence_len),]
            seq_y = valid_y[j:(j+sequence_len),]
            _, outputs = forward(seq_x, layers)
            valid_loss += mse(seq_y, outputs)

        print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")